In [1]:
import numpy as np
import pandas as pd

In [2]:
from subprocess import check_output
import os, subprocess
##print(os.listdir('C:\Users\hp')) 

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv 
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
np.random.seed(1234)

/home/lgriera/miniconda3/envs/keras-gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def data_power_consumption(path_to_dataset,
                           sequence_length=50,
                           ratio=1.0):
    
    max_values = ratio * 2049280
    
    with open(path_to_dataset) as f:
        data = csv.reader(f, delimiter=';')
        power = []
        nb_of_values = 0
        for line in data:
            try:
                power.append(float(line[2]))
                nb_of_values += 1 
            except ValueError:
                pass
            if nb_of_values >= max_values:
                break
                
    print("Data loaded from csv. Formatting...")
    
    result = []
    for index in range(len(power) - sequence_length):
        result.append(power[index: index+sequence_length])
    result = np.array(result)
    
    result_mean = result.mean()
    result -= result_mean
    print("Shift :", result_mean)
    print("Data :", result.shape)
    
    row = int(round(0.9 * result.shape[0]))
    train = result[:row, :]
    np.random.shuffle(train)
    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = result[row:, :-1]
    y_test = result[row:, -1]
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return [X_train, y_train, X_test, y_test]
        

In [5]:
def build_model():
    model = Sequential()
    layers = [1,50,100,1]
    
    model.add(LSTM(
        layers[1],
        input_shape=(None, layers[0]),
        return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))
    
    model.add(Dense(
        layers[3]))
    model.add(Activation("linear"))
    
    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop", metrics=['accuracy'])
    print ("Compilation Time: ", time.time() - start)
    return model

In [12]:
PATH = '/home/lgriera/Documents/ME-592x/Assignments/Assignment_4/'
file_name = 'household_power_consumption.txt'

In [13]:
def run_network(model=None, data=None):
    global_start_time = time.time()
    epochs = 1
    ratio = 0.5
    sequence_length = 50 
    #path_to_dataset = r'C:\Users\hp\household_power_consumption.txt'
    path_to_dataset = PATH + file_name
    
    if data is None:
        print('Loading data...')
        X_train, y_train, X_test, y_test = data_power_consumption(
            path_to_dataset, sequence_length, ratio)
    else:
            X_train, y_train. X_test, y_test = data
        
    print('\nData Loaded. Compiling...\n')
        
    if model is None:
            model = build_model()
    try:
            model.fit(
                X_train, y_train,
                batch_size=512, nb_epoch=epochs, validation_split=0.05)
            predicted = model.predict(X_test)
            predicted = np.reshape(predicted, (predicted.size,))
    except KerboardInterrupt:
             print( 'Training duration (s) : ', time.time() - global_start_time)
    return model, y_test, 0
        
    try:
            fig = plt.figure(0)
            ax = fig.add_subplot(111)
            ax.plot(y_test[:100],'r')
            plt.plot(predicted[:100],'b')
            plt.show()
    except Exception as e:
            print (str(e))
            print ('Training duration (s) : ', time.time() - global_start_time)
    
    return model, y_test, predicted
    
    

In [14]:
run_network()

Loading data...
Data loaded from csv. Formatting...
Shift : 1.1030964434554305
Data : (1024590, 50)

Data Loaded. Compiling...

Compilation Time:  0.019842147827148438


/home/lgriera/miniconda3/envs/keras-gpu/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 876024 samples, validate on 46107 samples
Epoch 1/1
876024/876024 [==============================] - 271s 310us/step - loss: 0.1003 - acc: 0.0000e+00 - val_loss: 0.0773 - val_acc: 0.0000e+00


(<keras.models.Sequential at 0x7f351f920c50>,
 array([1.33890356, 1.33090356, 1.33290356, ..., 0.22090356, 0.21690356,
        0.20090356]),
 0)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')